In [1]:
import os

from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import multiprocessing as mp
import shutil

from imblearn.keras import BalancedBatchGenerator
from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
img_size = 96
epochs = 75
batch_size = 32
testsplit = .2
targetx = 96
targety = 96
learning_rate = 0.0001
classes = 7
seed = 23
print(seed)


23


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter

model_name = 'SeqFemaleDominantmodel.keras'
# Define two different ImageDataGenerators with their respective directories
datagen1 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

datagen2 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

train_dir1 = 'KDEFMF/MaleNonDominantTraining'
train_dir2 = 'KDEFMF/FemaleDominantTraining'
val_dir1 = 'KDEFMF/MaleBalancedValidation'
val_dir2 = 'KDEFMF/FemaleBalancedValidation'

# Create two separate generators using different directories
train_generator1 = datagen1.flow_from_directory(
    train_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)
validation_generator1 = datagen1.flow_from_directory(
    val_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
)
validation_generator2 = datagen2.flow_from_directory(
    val_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
)

train_generator2 = datagen2.flow_from_directory(
    train_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

total_train_samples = len(train_generator1.filenames) + len(train_generator2.filenames)
total_validation_samples = len(validation_generator1.filenames) + len(validation_generator2.filenames)

# Combine labels from both generators
all_labels = list(train_generator1.classes) + list(train_generator2.classes)

# Get class indices (assuming they are the same for both generators)
class_indices = train_generator1.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# Count the number of occurrences of each class
class_counts = Counter(all_labels)

# Print the number of images per class
for class_index, count in class_counts.items():
    class_name = index_to_class[class_index]
    print(f"Class '{class_name}' has {count} images.")
    
def combined_generator(gen1, gen2, num_batches):
    while True:
        try:
            batch1_x, batch1_y = next(gen1)
            batch2_x, batch2_y = next(gen2)

            combined_x = np.concatenate((batch1_x, batch2_x), axis=0)
            combined_y = np.concatenate((batch1_y, batch2_y), axis=0)

            yield combined_x, combined_y
        except StopIteration as e:
            print("Generator exhausted or encountered an error:", e)
            break


Found 420 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 980 images belonging to 7 classes.
Class 'angry' has 200 images.
Class 'disgust' has 200 images.
Class 'fear' has 200 images.
Class 'happy' has 200 images.
Class 'neutral' has 200 images.
Class 'sad' has 200 images.
Class 'surprise' has 200 images.


In [4]:
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

model = Sequential()

# Custom Layers neccesarry to acheive high acurracy 
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten() (x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(512, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(256, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
predictions = Dense(classes, activation='softmax', kernel_initializer='random_uniform', bias_initializer='zeros')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(learning_rate=learning_rate)
loss = "categorical_crossentropy"

for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

In [5]:
# Define the number of batches you want from each generator
num_batches_per_gen = 100  # Adjust as needed

# Use the combined generator for training
combined_train_generator = combined_generator(train_generator1, train_generator2, num_batches_per_gen)

# Use the combined generator for validation
combined_validation_generator = combined_generator(validation_generator1, validation_generator2, num_batches_per_gen)


steps_per_epoch = total_train_samples // (2 * 32)
validation_steps = total_validation_samples // (2 * 32)

# Ensure correct optimizer and loss function
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Ensure ModelCheckpoint is correctly configured used for callback to save the best value based on val_accuracy
checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model with callbacks
history = model.fit(
    combined_train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=combined_validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    callbacks=[checkpoint]
)

Epoch 1/75
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.1572 - loss: 7.1582
Epoch 1: val_accuracy improved from -inf to 0.14062, saving model to SeqFemaleDominantmodel.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 826s 9s/step - accuracy: 0.1571 - loss: 7.1526 - val_accuracy: 0.1406 - val_loss: 5.2887
Epoch 2/75
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1972 - loss: 6.7824
Epoch 2: val_accuracy improved from 0.14062 to 0.14315, saving model to SeqFemaleDominantmodel.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - accuracy: 0.1971 - loss: 6.7761 - val_accuracy: 0.1431 - val_loss: 10.5241
Epoch 3/75
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.2143 - loss: 6.5603
Epoch 3: val_accuracy improved from 0.14315 to 0.15323, saving model to SeqFemaleDominantmodel.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 97s 5s/step - accuracy: 0.2149 - loss: 6.5544 - val_accuracy: 0.1532 - val_loss: 9.5787
Epoch 4/75
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.2825 - loss: 6.1707
Epoch 4: val_accuracy

In [6]:
model = tf.keras.models.load_model(model_name)

In [7]:
# Define the number of batches you want from each generator
num_batches_per_gen = 100  # Adjust as needed

# Use the combined generator for training
combined_train_generator = combined_generator(train_generator1, train_generator2, num_batches_per_gen)

# Use the combined generator for validation
combined_validation_generator = combined_generator(validation_generator1, validation_generator2, num_batches_per_gen)


steps_per_epoch = total_train_samples // (2 * 32)
validation_steps = total_validation_samples // (2 * 32)

In [8]:
# Collect predictions and true labels
y_true = []
y_pred = []

for batch_x, batch_y in combined_validation_generator:
    # Make predictions
    predictions = model.predict(batch_x)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(batch_y, axis=1)
    
    # Append to lists
    y_true.extend(true_classes)
    y_pred.extend(predicted_classes)
    
    # Stop if you have processed the entire validation set
    if len(y_true) >= total_validation_samples:
        break

# Generate classification report
report = classification_report(y_true, y_pred, target_names=validation_generator1.class_indices.keys())
print(report)

2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 279ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step
              precision    recall  f1-score   support

       angry       0.99      1.00      0.99        79
     disgust       1.00      0.96      0.98        85
        fear       0.89      1.00      0.94        75
       happy       1.00      0.95      0.97        78
     neutral       1.00      0.99      0.99        82
         sad       0.97      0.97      0.97        76
    surprise       1.00      0.98      0.99        85

    accuracy                           0.98       560
   macro avg       0.98      0.98      0.98       560
weighted avg       0.98      0.98      0.98       560



In [9]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Assuming you have the model and validation generators defined
# validation_generator1 and validation_generator2
# Also assuming the target_names are the same for both generators

# Function to generate and print classification report
def generate_classification_report(generator, generator_name):
    y_true = []
    y_pred = []

    for batch_x, batch_y in generator:
        # Generate predictions
        predictions = model.predict(batch_x)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = np.argmax(batch_y, axis=1)

        # Collect predictions and true labels
        y_true.extend(true_classes)
        y_pred.extend(predicted_classes)

        # Break if processed all samples
        if len(y_true) >= generator.samples:
            break

    # Generate and print the classification report
    target_names = list(generator.class_indices.keys())
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(f"Classification Report for {generator_name}:\n{report}")

# Generate and print classification reports for both validation generators
generate_classification_report(validation_generator1, "Validation Generator 1")
generate_classification_report(validation_generator2, "Validation Generator 2")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
Classification Report for Validation Generator 1:
              precision    recall  f1-score   support

       angry       1.00      1.00      1.00        40
     disgust       1.00      1.00      1.00        42
        fear       0.88      1.00      0.94        38
       happy       1.00      0.93      0.96        41
     neutral       1.00      0.98      0.99        43
         sad       1.00      1.00      1.00        38
    surprise       1.00      0.97      0.99        38

    accuracy                           0.98       280
   macro avg       0.98      0.98      0.98       280
weighted avg       0.98      0.98      0.98       280

1/

In [10]:
print(model_name)

SeqFemaleDominantmodel.keras


In [11]:
import numpy as np

def balanced_combination_generator(validation_generator1, validation_generator2, batch_size):
    class_indices = validation_generator1.class_indices
    num_classes = len(class_indices)

    while True:
        # Collect images and labels for a batch
        combined_images = []
        combined_labels = []

        # Determine how many samples to draw from each generator
        half_batch_size = batch_size // 2

        # Draw half batch from each generator
        for _ in range(half_batch_size):
            images1, labels1 = next(validation_generator1)
            images2, labels2 = next(validation_generator2)

            # Add to combined list
            combined_images.append(images1)
            combined_labels.append(labels1)
            combined_images.append(images2)
            combined_labels.append(labels2)

        # Concatenate all gathered images and labels
        combined_images = np.concatenate(combined_images, axis=0)
        combined_labels = np.concatenate(combined_labels, axis=0)

        # Shuffle the combined batch
        indices = np.arange(combined_images.shape[0])
        np.random.shuffle(indices)

        combined_images = combined_images[indices]
        combined_labels = combined_labels[indices]

        yield combined_images, combined_labels

# Use the custom generator in model evaluation
batch_size = 32
combined_validation_generator = balanced_combination_generator(
    validation_generator1, validation_generator2, batch_size=batch_size
)

# Evaluate the model using the combined validation generator
model.evaluate(combined_validation_generator, steps=validation_steps)

8/8 ━━━━━━━━━━━━━━━━━━━━ 67s 8s/step - accuracy: 0.9747 - loss: 1.0195


[1.0209392309188843, 0.974094569683075]

In [12]:
combined_validation_generator_samples = combined_validation_generator.samples

AttributeError: 'generator' object has no attribute 'samples'